In [6]:
from pyodbc import Cursor
import requests
import os
from dotenv import load_dotenv
from shared.get_connections import get_db_mongo
load_dotenv("config.env")

MONGODB_PRODUCT_COLLECTION = os.environ.get('MONGODB_PRODUCT_COLLECTION')
MONGODB_CATALOG_COLLECTION = os.environ.get('MONGODB_CATALOG_COLLECTION')
DOTNET_BULK_ADD_PROVIDER_REQUIREMENTS_ENDPOINT = os.environ.get('DOTNET_BULK_ADD_PROVIDER_REQUIREMENTS_ENDPOINT')
db = get_db_mongo()
product_collection = db[MONGODB_PRODUCT_COLLECTION]
catalog_collection = db[MONGODB_CATALOG_COLLECTION]
cursor_products = product_collection.find({})
cursor_catalogs = catalog_collection.find({})

post_request_payload = []

2024-01-16 11:22:52,191 - INFO - Connected to MongoDB database


### This script is to create provider for all product model in mongodb from catalog

In [7]:
def group_catalog_by_step(cursor_catalogs: Cursor, group_size: int = 3) -> list(list()):
    """Group catalogs by step. Return a list of (group or list of catalogs)."""
    # set up
    grouped_catalogs_list = [] # result list
    group = [] # group of 3 catalogs is a list
    step = 0

    #each iter, add catalog to group
    #after 3 iter, add group to grouped_catalogs_list and reset group
    for catalog in cursor_catalogs:
        step += 1
        group.append(catalog)
        if step == group_size:
            grouped_catalogs_list.append(group)
            group = []
            step = 0

    # if the last group is not empty, add it to grouped_catalogs_list (step is not == 3)
    if step != 0:
        grouped_catalogs_list.append(group)
    return grouped_catalogs_list

group_list = group_catalog_by_step(cursor_catalogs)
for group in group_list:
    print(group)

[{'_id': '424cb419-aa6a-4a07-9a83-fbf167864554', 'CatalogSubCatalogs': [], 'CatalogName': 'HOME IMPROVEMENT AND GARDENING'}, {'_id': 'db0288e7-4bb4-4ccb-b825-df9108d9e71b', 'CatalogSubCatalogs': [{'_id': '0b17000d-af19-455f-b0e4-71ddd8da113d', 'SubCatalogName': 'ADDED VITAMINS WATER'}, {'_id': 'db6d3c8e-8ec2-4607-ae75-437c35aac3dc', 'SubCatalogName': 'NATURAL WATER'}], 'CatalogName': 'WATER'}, {'_id': 'b3f4697b-ee01-4023-9cde-82ce5fe4a6ca', 'CatalogSubCatalogs': [{'_id': '77f1e026-3b82-4238-93e4-3d0f70779714', 'SubCatalogName': 'MULTI PURPOSE CLEANER'}, {'_id': '34d9543f-357d-48e6-a3d1-a2f6954ca0e9', 'SubCatalogName': 'WASHING POWDER'}, {'_id': '726d3b4c-225e-4bff-8ab8-f9c720aece3e', 'SubCatalogName': 'WC'}], 'CatalogName': 'HOME CARE'}]
[{'_id': '1ebd3790-886b-43d3-9cf0-9117ff79e985', 'CatalogSubCatalogs': [], 'CatalogName': 'BOXES AND BAGS'}, {'_id': 'bfa80fd9-713a-47bf-b0a1-c8f4976e0b2d', 'CatalogSubCatalogs': [{'_id': '222304f9-16ce-47bf-9369-b0cc4574a5ff', 'SubCatalogName': 'CUTTI

In [8]:
# list (of all group) of list of each group catalog ids
get_catalog_ids_list_lambda = lambda group: [catalog['_id'] for catalog in group]
catalog_ids_a_group_list = [get_catalog_ids_list_lambda(group) for group in group_list]
print(catalog_ids_a_group_list)

[['424cb419-aa6a-4a07-9a83-fbf167864554', 'db0288e7-4bb4-4ccb-b825-df9108d9e71b', 'b3f4697b-ee01-4023-9cde-82ce5fe4a6ca'], ['1ebd3790-886b-43d3-9cf0-9117ff79e985', 'bfa80fd9-713a-47bf-b0a1-c8f4976e0b2d', 'a58e80d2-0dc3-4127-84da-2501316ed1f4'], ['6f6d708a-5575-4532-ac2e-92a9d2c2f72e', '59958d90-e0ff-4fb3-9451-32f76076f03e', '7c295f25-d5b0-4032-9288-d7c35bb06a57'], ['da7493e6-b0ce-473f-9953-99cdd82d164a', '1a88d3f8-1aea-4563-9eca-1dfee6f08f13', 'c3371be9-4fda-4290-8e74-59a91379f882'], ['1917a3f7-f787-462d-9d35-c8e5c5a3627e', '462e5e7c-e7b4-4f60-aeb8-e3ff76a288fa', 'a7c3b0f5-d729-4875-9790-f452285cb17c'], ['cb564bd5-257a-4b48-b125-5931abd991b7', '02fb57d0-eb55-4f9e-8738-8f126fac7348', 'f32e2fb4-6cfa-41bc-bdd8-f742488f52b0'], ['1def499d-adcf-4469-aa5f-56c760fa4958', 'c51fc97e-809a-43e6-80ee-7746f41f050d', '65411e8e-24dc-478e-9de9-1953328d5979'], ['b6fb5318-c632-43e9-9719-4b4c972e3c67', 'ff893325-de64-4956-a2ea-1f0b630d60ab', '1383b8d7-c1d0-415f-a9de-855fc7b96321'], ['b39d9cf8-c0c5-43cd-b0

In [10]:
def get_catalog_subcatalog_id_list(catalog: object) -> list():
    """Get list of sub catalog ids of each catalog."""
    catalog_sub_catalog_id_list = []
    for sub_catalog in catalog['CatalogSubCatalogs']:
        catalog_sub_catalog_id_list.append(sub_catalog['_id'])
    return catalog_sub_catalog_id_list

catalog = catalog_collection.find_one({'_id': 'db0288e7-4bb4-4ccb-b825-df9108d9e71b'})
catalog_sub_catalog_id_list = get_catalog_subcatalog_id_list(catalog)
print(catalog_sub_catalog_id_list)

['0b17000d-af19-455f-b0e4-71ddd8da113d', 'db6d3c8e-8ec2-4607-ae75-437c35aac3dc']


In [11]:
# Find all product of belong to a subcatalog (HAVE subcatalog_id) to list of object (product id, product model id, product business key)
def get_product_meta_list_a_subcatalog(subcatalog_id: str) -> list():
    product_meta_list_a_subcatalog = []
    product_cursor_of_subcatalog = product_collection.find({'SubCatalogId': subcatalog_id})
    for product in product_cursor_of_subcatalog:
        product_meta_list_a_subcatalog.append({
            'productId': product['_id'],
            'productModelId': product['ProductModels'][0]['_id'],
            'businessKey': product['BusinessKey'],
            'unitRequestPrice': 0
        })
    return product_meta_list_a_subcatalog

# From get_product_meta_list_a_subcatalog, get list of product meta of all subcatalogs of catalog in a group.
def get_product_meta_list_a_group(subcatalog_id_list_a_group: str) -> list():
    if subcatalog_id_list_a_group == []:
        return []
    product_meta_list_a_group = []
    for subcatalog_id in subcatalog_id_list_a_group:
        product_meta_list_a_subcatalog = get_product_meta_list_a_subcatalog(subcatalog_id)
        product_meta_list_a_group.extend(product_meta_list_a_subcatalog)
    return product_meta_list_a_group

# Get list of sub catalog ids of every catalog in a group
def get_subcatalog_id_list_a_group(group: list()) -> list(list()):
    subcatalog_id_list_a_group = []
    for catalog in group:
        catalog_subcatalog_id_list = get_catalog_subcatalog_id_list(catalog)
        subcatalog_id_list_a_group.extend(catalog_subcatalog_id_list)
    return subcatalog_id_list_a_group

# group is a list of catalog objects
# group_list is a list of all groups
for group in group_list:
    subcatalog_id_list_a_group = get_subcatalog_id_list_a_group(group)
    # print(subcatalog_id_list_a_group)
    product_meta_list_a_group = get_product_meta_list_a_group(subcatalog_id_list_a_group)
    print(product_meta_list_a_group)

[{'productId': 'a490f4e0-25bc-4e7c-83b9-c5806e7312a5', 'productModelId': '4d2aac8a-e35e-4cfc-a7fd-25b539f90448', 'businessKey': 'ac141966-5b6e-4c9d-96f0-6b1351105d40', 'unitRequestPrice': 0}, {'productId': '03aed8c9-2643-4c97-8940-85fd0ae347f6', 'productModelId': 'caadb567-cbdf-41ed-8ec2-9d9766424d32', 'businessKey': 'dfd9682b-ac57-498e-90a2-536449cd651d', 'unitRequestPrice': 0}, {'productId': 'acc94c65-b2fc-4966-bf4a-3b7e3a60f72b', 'productModelId': '82bceb35-47fc-4c56-a67b-88bdb6621c3f', 'businessKey': '7efc4829-63e1-4368-b5b7-8f555b4d3576', 'unitRequestPrice': 0}, {'productId': 'b12b85bb-6de1-4179-8791-13b3c29d9fc5', 'productModelId': 'f7326036-a116-4368-b3f9-23edd5b1c092', 'businessKey': '4ba82d02-289f-4817-8ce2-069b51c5da87', 'unitRequestPrice': 0}, {'productId': 'a2f27267-6392-4e08-8d9e-563e6c1f58d7', 'productModelId': '0110c081-13bd-45c1-bd09-d63d2a88978b', 'businessKey': '33c15056-c38c-42ad-bb83-03f2e605c1f7', 'unitRequestPrice': 0}, {'productId': '32abd085-75a1-4211-af06-69d5f

### Now is for real

In [2]:
def group_catalog_by_step(cursor_catalogs: Cursor, group_size: int = 3) -> list(list()):
    """Group catalogs by step. Return a list of (group or list of catalogs)."""
    grouped_catalogs_list = []
    group = [] # group of 3 catalogs is a list
    step = 0

    #each iter, add catalog to group
    #after 3 iter, add group to grouped_catalogs_list and reset group
    for catalog in cursor_catalogs:
        step += 1
        group.append(catalog)
        if step == group_size:
            grouped_catalogs_list.append(group)
            group = []
            step = 0

    # if the last group is not empty, add it to grouped_catalogs_list (step is not == 3)
    if step != 0:
        grouped_catalogs_list.append(group)
    return grouped_catalogs_list

get_catalog_ids_list_in_group_lambda = lambda group: [catalog['_id'] for catalog in group]
get_catalog_names_list_in_group_lambda = lambda group: [catalog['CatalogName'] for catalog in group if catalog['CatalogName'] != None]

def get_catalog_subcatalog_id_list(catalog: object) -> list():
    """Get list of sub catalog ids of each catalog."""
    catalog_sub_catalog_id_list = []
    for sub_catalog in catalog['CatalogSubCatalogs']:
        catalog_sub_catalog_id_list.append(sub_catalog['_id'])
    return catalog_sub_catalog_id_list

def get_product_meta_list_a_subcatalog(subcatalog_id: str) -> list():
    """
        Find all product of belong to a subcatalog (HAVE subcatalog_id) 
        to list of object (product id, product model id, product business key)
    """
    product_meta_list_a_subcatalog = []
    product_cursor_of_subcatalog = product_collection.find({'SubCatalogId': subcatalog_id})
    for product in product_cursor_of_subcatalog:
        product_meta_list_a_subcatalog.append({
            'productId': product['_id'],
            'productModelId': product['ProductModels'][0]['_id'],
            'businessKey': product['BusinessKey'],
            'unitRequestPrice': 0
        })
    return product_meta_list_a_subcatalog

def get_subcatalog_id_list_a_group(group: list()) -> list(list()):
    """Get list of sub catalog ids of every catalog in a group."""
    subcatalog_id_list_a_group = []
    for catalog in group:
        catalog_subcatalog_id_list = get_catalog_subcatalog_id_list(catalog)
        subcatalog_id_list_a_group.extend(catalog_subcatalog_id_list)
    return subcatalog_id_list_a_group  

def get_product_meta_list_a_group(subcatalog_id_list_a_group: str) -> list():
    """From get_product_meta_list_a_subcatalog, get list of product meta of all subcatalogs of catalog in a group."""
    if subcatalog_id_list_a_group == []:
        return []
    product_meta_list_a_group = []
    for subcatalog_id in subcatalog_id_list_a_group:
        product_meta_list_a_subcatalog = get_product_meta_list_a_subcatalog(subcatalog_id)
        product_meta_list_a_group.extend(product_meta_list_a_subcatalog)
    return product_meta_list_a_group 

In [3]:
from uuid import uuid4

post_request_payload = []
cursor_catalogs = catalog_collection.find({})
group_catalogs_list = group_catalog_by_step(cursor_catalogs)
for group in group_catalogs_list:
    available_provider_catalog_ids = get_catalog_ids_list_in_group_lambda(group)
    catalog_names_list_a_group = get_catalog_names_list_in_group_lambda(group)
    provider_name_a_group = ' - '.join(catalog_names_list_a_group)    
    provider_name_a_group = 'PROVIDER FOR ' + provider_name_a_group
    subcatalog_id_list_a_group = get_subcatalog_id_list_a_group(group)
    product_meta_list_a_group = get_product_meta_list_a_group(subcatalog_id_list_a_group)
    if product_meta_list_a_group == []:
        continue

    # print(available_provider_catalog_ids)
    # print(subcatalog_id_list_a_group)
    # print(catalog_names_list_a_group)
    # print(provider_name_a_group)
    # print(product_meta_list_a_group)

    post_request_payload.append({
        'providerName': provider_name_a_group,
        'minPriceToBeAccepted': 0,
        'minQuantityToBeAccepted': 0,
        'providerBusinessKey': uuid4().__str__(),
        'revision': 0,
        'isUsed': True,
        'availableProviderCatalogIds': available_provider_catalog_ids,
        'availableStockItemRequestMetas': product_meta_list_a_group
    })

In [4]:
response = requests.post(DOTNET_BULK_ADD_PROVIDER_REQUIREMENTS_ENDPOINT, json=post_request_payload)
print("Request:", response.request.body)
print("Response content:", response.content)
print("Status code:", response.status_code)
print("Response:", response.reason)

Request: b'[{"providerName": "PROVIDER FOR HOME IMPROVEMENT AND GARDENING - WATER - HOME CARE", "minPriceToBeAccepted": 0, "minQuantityToBeAccepted": 0, "providerBusinessKey": "ab1f7e30-fcad-46b1-b311-a2cc0500abad", "revision": 0, "isUsed": true, "availableProviderCatalogIds": ["424cb419-aa6a-4a07-9a83-fbf167864554", "db0288e7-4bb4-4ccb-b825-df9108d9e71b", "b3f4697b-ee01-4023-9cde-82ce5fe4a6ca"], "availableStockItemRequestMetas": [{"productId": "a490f4e0-25bc-4e7c-83b9-c5806e7312a5", "productModelId": "4d2aac8a-e35e-4cfc-a7fd-25b539f90448", "businessKey": "ac141966-5b6e-4c9d-96f0-6b1351105d40", "unitRequestPrice": 0}, {"productId": "03aed8c9-2643-4c97-8940-85fd0ae347f6", "productModelId": "caadb567-cbdf-41ed-8ec2-9d9766424d32", "businessKey": "dfd9682b-ac57-498e-90a2-536449cd651d", "unitRequestPrice": 0}, {"productId": "acc94c65-b2fc-4966-bf4a-3b7e3a60f72b", "productModelId": "82bceb35-47fc-4c56-a67b-88bdb6621c3f", "businessKey": "7efc4829-63e1-4368-b5b7-8f555b4d3576", "unitRequestPric